<a href="https://colab.research.google.com/github/casjunior93/Tera---Desafio-de-classificacao/blob/main/Refeito_Tera_Desafio_de_Classifica%C3%A7%C3%A3o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Desafio de Detecção e Redução de Fraudes no Setor Financeiro

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


O desafio é propor uma solução para detecção e análise de fraudes que possam reduzir os riscos da empresa de crédito em que trabalho e garantir margens saudáveis.

Link do dataset: https://drive.google.com/file/d/1zjK8zQK5zvhR_r2chWI5dCjeOwASlPfb/view

Vou começar importando as bibliotecas iniciais:

In [26]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

Agora importo os dados:

In [3]:
df = pd.read_csv('/content/drive/MyDrive/Tera/DESAFIO CLASSIFICAÇÃO/fraud_detection_dataset.csv')

In [4]:
df.sample(10)

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
1281066,135,PAYMENT,22770.72,C2023393770,82564.00,59793.28,M1152619044,0.00,0.00,0,0
461103,19,TRANSFER,685429.60,C1492248963,0.00,0.00,C1734649556,5337201.53,6022631.13,0,0
5060894,354,CASH_OUT,72858.33,C1889408849,194145.68,121287.35,C1374705406,2008357.93,2081216.26,0,0
241130,14,PAYMENT,8925.14,C1836976195,0.00,0.00,M891184211,0.00,0.00,0,0
6057516,500,PAYMENT,7653.76,C1532242437,0.00,0.00,M915812644,0.00,0.00,0,0
6295130,670,PAYMENT,3972.03,C1728174376,37211.00,33238.97,M2056448717,0.00,0.00,0,0
3607335,264,CASH_IN,58212.34,C649216031,1803333.92,1861546.25,C1808226540,135271.30,77058.96,0,0
910348,43,CASH_IN,69884.98,C91975976,6634267.46,6704152.44,C1311565766,1411675.89,1341790.91,0,0
2489185,204,CASH_OUT,68277.85,C61249607,0.00,0.00,C1386086421,86056.58,154334.43,0,0
1745297,161,CASH_OUT,7425.52,C830518750,0.00,0.00,C989803368,6579183.69,6586609.21,0,0


## Conhecendo as features deste dataset

* `step` - Passo: representa o total de horas transcorrido desde o início da simulação. Esta feature vai variar entre 1 e 744 (30 dias);

* `type` - Tipo: tipo de transação (depósito, saque, débito, pagamento e transferência);

* `amount` - Quantia: total que foi transacionado;

* `nameOrig` - ClienteOrigem: cliente que iniciou a transação

* `oldbalanceOrg` - SaldoInicialOrigem: saldo da conta de origem antes da transação;

* `newbalanceOrig` - SaldoFinalOrigem: saldo da conta de origem após a transação;

* `nameDest` - ClienteDestino: cliente de destino da transação;

* `oldbalanceDest` - SaldoInicialDestino: saldo da conta de destino antes da transação;

* `newbalanceDest` - SaldoFinalDestino: saldo da conta de destino após a transação;

* `isFraud` - ÉFraude: flag que define se a transação é fraudulenta ou não. Nesta simulação o objetivo da fraude é assumir a conta do usuário, esvaziá-la transferindo para outra conta e então sacando o dinheiro.

* `isFlaggedFraud` - SinalizadaComoFraude: automaticamente marcadas pelo banco como fraude por tentarem transferir mais de 200.000 em uma única transação.

## Análise exploratória

Primeiro vou renomear as tabelas para facilitar a análise.

In [7]:
df = df.rename(
    columns={"step":"totalHoras", "type": "tipoTransacao", "amount": "quantia",
             "nameOrig": "clienteOrigem", "oldbalanceOrg": "saldoInicialOrigem",
             "newbalanceOrig": "saldoFinalOrigem", "nameDest": "clienteDestino",
             "oldbalanceDest": "saldoInicialDestino", "newbalanceDest": "saldoFinalDestino",
             "isFraud": "ehFraude", "isFlaggedFraud": "sinalizadaComoFraude"

             })
df.head(3)

,totalHoras,tipoTransacao,quantia,clienteOrigem,saldoInicialOrigem,saldoFinalOrigem,clienteDestino,saldoInicialDestino,saldoFinalDestino,ehFraude,sinalizadaComoFraude
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0


Análise descritiva:

In [10]:
df.describe(include='all')

,totalHoras,tipoTransacao,quantia,clienteOrigem,saldoInicialOrigem,saldoFinalOrigem,clienteDestino,saldoInicialDestino,saldoFinalDestino,ehFraude,sinalizadaComoFraude
count,6.362620e+06,6362620,6.362620e+06,6362620,6.362620e+06,6.362620e+06,6362620,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06
unique,NaN,5,NaN,6353307,NaN,NaN,2722362,NaN,NaN,NaN,NaN
top,NaN,CASH_OUT,NaN,C1902386530,NaN,NaN,C1286084959,NaN,NaN,NaN,NaN
freq,NaN,2237500,NaN,3,NaN,NaN,113,NaN,NaN,NaN,NaN
mean,2.433972e+02,NaN,1.798619e+05,NaN,8.338831e+05,8.551137e+05,NaN,1.100702e+06,1.224996e+06,1.290820e-03,2.514687e-06
std,1.423320e+02,NaN,6.038582e+05,NaN,2.888243e+06,2.924049e+06,NaN,3.399180e+06,3.674129e+06,3.590480e-02,1.585775e-03
min,1.000000e+00,NaN,0.000000e+00,NaN,0.000000e+00,0.000000e+00,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.560000e+02,NaN,1.338957e+04,NaN,0.000000e+00,0.000000e+00,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,2.390000e+02,NaN,7.487194e+04,NaN,1.420800e+04,0.000000e+00,NaN,1.327057e+05,2.146614e+05,0.000000e+00,0.000000e+00
75%,3.350000e+02,NaN,2.087215e+05,NaN,1.073152e+05,1.442584e+05,NaN,9.430367e+05,1.111909e+06,0.000000e+00,0.000000e+00


Informações:
* Tenho 5 tipos de transação e o tipoTransacao mais frequente é CASH_OUT, com 2237500 transações.
* O cliente destino C1286084959 recebeu 113 transações no período.

Verificando a distribuição dos dados para analisar se são simétricos:

In [14]:
df.skew()

totalHoras                0.375177
quantia                  30.993949
saldoInicialOrigem        5.249136
saldoFinalOrigem          5.176884
saldoInicialDestino      19.921758
saldoFinalDestino        19.352302
ehFraude                 27.779538
sinalizadaComoFraude    630.603629
dtype: float64

A análise de assimetria indica que as variáveis apresentam uma assimetria positiva, ou seja, elas contém mais valores acima da média. 